In [1]:
import pandas as pd
import numpy as np

In [25]:
exp_2022 = pd.read_csv("..\Dados\COMEX\EXP_2022_MUN.csv", delimiter=";")
agropecuaria = pd.read_excel(r"..\Dados\COMEX\sh4_agropecuaria.xlsx")
tabela_final = pd.read_csv(r"..\Dados\Tabela_final\tabela_final.csv")

In [19]:
exp_2022

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB
0,2022,4,9403,845,PR,4101002,53482,47615
1,2022,10,8501,386,SC,4208906,220899,1348091
2,2022,9,1902,580,ES,3205309,564,2264
3,2022,1,6206,63,SC,4216602,85,6642
4,2022,2,8501,249,SC,4208906,2942192,12155328
...,...,...,...,...,...,...,...,...
1087126,2022,9,2201,445,SP,3448500,1260,328
1087127,2022,7,9603,586,RS,4304804,24,350
1087128,2022,9,8527,239,PR,4120804,234,34731
1087129,2022,1,203,767,PA,1506807,30,226


In [31]:
exp_2022 = exp_2022.merge(agropecuaria, how="inner", left_on="SH4", right_on="Código SH4")
exp_2022.drop(columns=["Código SH4"], inplace=True)

In [32]:
exp_2022

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,Descrição do produto exportado,Denominação nesse estudo
0,2022,9,5201,455,MT,5107875,1114253,2071800,"Algodão, não cardado nem penteado.",Algodão
1,2022,2,5201,827,MT,5107875,5938815,10428103,"Algodão, não cardado nem penteado.",Algodão
2,2022,2,5201,365,MT,5103502,1047269,1719167,"Algodão, não cardado nem penteado.",Algodão
3,2022,11,5201,858,MT,5107875,4691205,8303458,"Algodão, não cardado nem penteado.",Algodão
4,2022,3,5201,160,MA,2101400,170943,367305,"Algodão, não cardado nem penteado.",Algodão
...,...,...,...,...,...,...,...,...,...,...
121579,2022,8,4812,580,SP,3452205,2,178,"Blocos e chapas, filtrantes, de pasta de papel.",Papel
121580,2022,11,4812,493,SP,3430706,460,3989,"Blocos e chapas, filtrantes, de pasta de papel.",Papel
121581,2022,11,4812,850,SP,3443402,13,4616,"Blocos e chapas, filtrantes, de pasta de papel.",Papel
121582,2022,12,4812,43,ES,3205309,750,8000,"Blocos e chapas, filtrantes, de pasta de papel.",Papel


In [34]:
exp_2022 = exp_2022.groupby(["CO_MUN", "Denominação nesse estudo"]).agg({"VL_FOB":"mean", "KG_LIQUIDO":"mean"}).reset_index()

In [36]:
exp_2022_sertao = exp_2022[exp_2022["CO_MUN"].isin(tabela_final["IBGE7"])]

In [37]:
exp_2022_sertao

,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
394,2112209,Carne Bovina,1.117418e+05,3.003060e+04
396,2200400,Farelo de soja,4.730000e+05,9.302440e+05
397,2200400,Farelo/Farinha e Resíduos Sólidos,4.730000e+05,9.302440e+05
402,2201903,Algodão,1.672380e+05,7.943500e+04
403,2201903,Milho em grãos,4.544914e+06,1.591351e+07
...,...,...,...,...
721,2929909,Café em grãos,7.587000e+03,1.440000e+03
726,2933307,Café em grãos,7.196338e+05,1.754201e+05
871,3135100,Carne Bovina,5.370012e+05,1.065353e+05
934,3151206,Milho/Soja em grãos,6.885123e+06,1.182952e+07
